In [1]:
import torch
from pathlib import Path
import pandas as pd
import numpy as np
import cv2

In [2]:
model_name = "02_unet_custom_DataParallel.zip"
GPU = True
if GPU:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
path = Path("../")
data_path = path / "data"
test_images_path = data_path / "test_images"

In [4]:
test_df = pd.read_csv(data_path / "test.csv")
test_df['path'] = test_df.id.apply(lambda x: f'{str(test_images_path)}/{x}.tiff')
test_df.head()

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,path
0,10078,spleen,Hubmap,2023,2023,0.4945,4,../data/test_images/10078.tiff


In [5]:
def mask2rle(mask):
    '''
    mask: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def calc_mask(tensor, model):
    '''
    tensor: tensor of shape (1, 3, H, W)
    Returns mask of shape (H, W)
    '''
    if len(tensor.shape) == 3:
        tensor = tensor.unsqueeze(0)
    model.eval()
    with torch.no_grad():
        if GPU:
            tensor = tensor.to(device)
            print(f"tensor device: {tensor.device}")
        output = model(tensor).squeeze(0)
        pred_mask = torch.argmax(output, axis=0)
    return pred_mask.detach().cpu().numpy()

def calc_mask_from_image_path(img_path, model):
    '''
    img_path: path to image
    model: model to use for inference
    Returns mask of shape (H, W)
    '''
    img = cv2.imread(img_path, cv2.COLOR_BGR2RGB) / 255.0
    img_tensor = torch.from_numpy(img.astype(np.float32)).float().permute(2, 0, 1).unsqueeze(0)
    return calc_mask(img_tensor, model)

def calc_rle_from_image_path(img_path, model):
    '''
    img_path: path to image
    model: model to use for inference
    Returns rle string
    '''
    mask = calc_mask_from_image_path(img_path, model)
    return mask2rle(mask)

In [6]:
model_path = path / "models" / model_name
model = torch.jit.load(model_path)
if GPU:
    model = model.to(device)
    print("Using GPU")

Using GPU


In [7]:
test_df['rle'] = test_df.apply(lambda x: calc_rle_from_image_path(x.path, model), axis=1)
test_df.head()

tensor device: cuda:0


,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,path,rle
0,10078,spleen,Hubmap,2023,2023,0.4945,4,../data/test_images/10078.tiff,8305 4 9672 3 10321 13 10765 10 10983 6 11692 ...


In [8]:
submision_df = test_df[['id', 'rle']]
submision_df.head()

,id,rle
0,10078,8305 4 9672 3 10321 13 10765 10 10983 6 11692 ...


In [9]:
submision_df.to_csv(data_path / "submision.csv", index=False)